In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable #自動微分用
from torch.nn.functional import cross_entropy

import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

import torchvision
import torchvision.transforms as transforms


import matplotlib.pyplot as plt
import numpy as np

import numpy as np
import pandas as pd
from scipy.special import erfinv

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.preprocessing import StandardScaler
import lightgbm

from logging import getLogger
from tqdm import tqdm_notebook as tqdm
import argparse
import datetime
import random
from itertools import chain
import pickle
import warnings
from matplotlib import pyplot as plt
import seaborn as sns

import gc
import sys
from multiprocessing import Pool
sys.path.append('../')

from tools.my_logging import logInit
from tools.feature_tools import feature_engineering
from tools.objective_function import weighted_multi_logloss, lgb_multi_weighted_logloss, wloss_objective, wloss_metric, softmax, calc_team_score
from tools.model_io import save_models, load_models
from tools.fold_resampling import get_fold_resampling_dict

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

/home/naoya.taguchi/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
NTHREAD = 62
random.seed(71)
np.random.seed(42)
torch.manual_seed(71)
%load_ext autoreload
%autoreload 2

In [3]:
class plasticcNet(nn.Module):
    def __init__(self, input_unit_num, mlp_unit_nums, band_cnn_idxes=None,
                init_do_rate=0., middle_do_rate=0.5, last_do_rate=0.25):
        super(plasticcNet, self).__init__()
        self.activation = torch.nn.ELU()
        self.cnn_activation = torch.nn.ELU()
        self.cnn_dropout = torch.nn.Dropout(p=0.2)
        self.cnn_dropout2 = torch.nn.Dropout(p=0.2)
        self.band_cnn_idxes = band_cnn_idxes
        self.init_do_rate = init_do_rate
        self.middle_do_rate = middle_do_rate
        self.last_do_rate = last_do_rate
         
        # band 軽特徴量間の幾何的構造に基づく関係性を捉える cnn 
        # 1st では poolig せず、second で　pooling する？
        self.maxpool1 = nn.MaxPool1d(kernel_size=3, stride=1)
        self.maxpool2 = nn.MaxPool1d(kernel_size=2, stride=1)
        self.band_conv_nets1 = nn.ModuleList()
        self.band_conv_nets2 = nn.ModuleList()
        self.band_batch_norm1 = nn.ModuleList()
        self.band_batch_norm2 = nn.ModuleList()
        #self.band_dropout1 = nn.ModuleList()
        #self.band_dropout2 = nn.ModuleList()
        if self.band_cnn_idxes:
            for band_cnn_idx in band_cnn_idxes:
                self.band_conv_nets1.append(
                    nn.Conv1d(
                        in_channels=1,
                        out_channels=1,
                        kernel_size=4,
                        stride=1)
                )
                #self.band_batch_norm1.append(nn.LayerNorm(6))
                self.band_batch_norm1.append(nn.BatchNorm1d(1))
                #self.band_dropout1.append()
                self.band_conv_nets2.append(
                    nn.Conv1d(
                        in_channels=1,
                        out_channels=1,
                        kernel_size=3,
                        stride=1)
                )
                #self.band_batch_norm2.append(nn.LayerNorm(4))
                self.band_batch_norm2.append(nn.BatchNorm1d(1))
                #self.band_dropout2.append()
            None
       
        # mlp を組む
        # 一層目の unit num を作る
        if self.band_cnn_idxes:
            self.unpacked_band_cnn_idxes = list(chain.from_iterable(self.band_cnn_idxes))
            #past_unit_num = input_unit_num + len(self.unpacked_band_cnn_idxes) // 6 * 4 - len(self.unpacked_band_cnn_idxes) # only 1 layer
            #past_unit_num = input_unit_num + len(self.unpacked_band_cnn_idxes) // 6 * 6 - len(self.unpacked_band_cnn_idxes) # only 1 layer
            self.cnn_out_unit_num = len(self.unpacked_band_cnn_idxes) // 6 * 4
            self.cnn_bn = nn.LayerNorm(self.cnn_out_unit_num)
            #self.cnn_bn = nn.BatchNorm1d(self.cnn_out_unit_num)
            past_unit_num = input_unit_num + self.cnn_out_unit_num - len(self.unpacked_band_cnn_idxes) # w/o pooling
            #past_unit_num = input_unit_num + len(self.unpacked_band_cnn_idxes) // 6 * 3 - len(self.unpacked_band_cnn_idxes)
        else:
            past_unit_num = input_unit_num
        self.dense_nets = nn.ModuleList()
        self.batch_norms = nn.ModuleList()
        self.dropouts = nn.ModuleList()
        for i, mlp_unit_num in enumerate(mlp_unit_nums):
            self.dense_nets.append(nn.Linear(past_unit_num, mlp_unit_num))
            self.batch_norms.append(nn.BatchNorm1d(mlp_unit_num))
            if i == 0:
                p = self.init_do_rate
            elif i == len(mlp_unit_nums) - 1:
                p = self.last_do_rate
            else:
                p = self.middle_do_rate
            self.dropouts.append(nn.Dropout(p=p))
            past_unit_num = mlp_unit_num
            
    def forward(self, x):
        batch_size = x.shape[0]
        x_cnns = []
        if len(self.band_conv_nets1) > 0:
            for zipped in zip(self.band_cnn_idxes, 
                            self.band_conv_nets1, self.band_batch_norm1, 
                            self.band_conv_nets2, self.band_batch_norm2):
                band_cnn_idx, band_conv_net1, band_batch_norm1, band_conv_net2, band_batch_norm2 = zipped
                x_cnn = x[:, band_cnn_idx]
                x_cnn = torch.cat([x_cnn, x_cnn[:,:3]], dim=1) # mod
                x_cnn = x_cnn.view(batch_size, 1, -1) 
                x_cnn = band_conv_net1(x_cnn) 
                x_cnn = self.cnn_activation(x_cnn)
                x_cnn = band_batch_norm1(x_cnn)
                x_cnn = self.cnn_dropout(x_cnn)
                #x_cnn = self.maxpool1(x_cnn)
                x_cnn = band_conv_net2(x_cnn) 
                x_cnn = self.cnn_activation(x_cnn)
                #x_cnn = band_batch_norm2(x_cnn)
                #x_cnn = self.cnn_dropout(x_cnn)
                #x_cnn = self.maxpool2(x_cnn)
                x_cnns.append(x_cnn)

            x_cnns = torch.cat(x_cnns, dim=1).view(batch_size, -1) # concat して channel を押しつぶす
            x_cnns = self.cnn_bn(x_cnns)
            x_cnns = self.cnn_dropout2(x_cnns)
            x = np.delete(x, self.unpacked_band_cnn_idxes, axis=1) # drop used idxes  self.unpacked....
            x = torch.cat([x, x_cnns], dim=1) 
        else:
            x = x
        for i, nets in enumerate(list(zip(self.dense_nets, self.batch_norms, self.dropouts))):
            dense_net, batch_norm, dropout = nets
            if i < len(self.dense_nets) - 1:
                x = dropout(x)
                x = dense_net(x)
                x = self.activation(x)
                x = batch_norm(x)
            else:
                x = dropout(x)
                x =  dense_net(x)
                #x = batch_norm(x)
                #x = dropout(x)
        return x

In [4]:
class MKRankGaussScalar(object):
    """usage: 
    rgs = RankGaussScalar()
    rgs.fit(df_X)
    df_X_converted = rgs.transform(df_X)
    df_X_test_converted = rgs.transform(df_X_test)
    """
    def __init__(self):
        self.fit_done = False

    def rank_gauss(self, x):
        N = x.shape[0]
        temp = x.argsort()
        rank_x = temp.argsort() / N
        rank_x -= rank_x.mean()
        rank_x *= 2
        efi_x = erfinv(rank_x)
        efi_x -= efi_x.mean()
        return efi_x

    def fit(self, df_x):
        """
        df_x: fitting対象のDataFrame
        """
        self.train_unique_rankgauss = {}
        self.target_cols = np.sort(df_x.columns)
        for c in self.target_cols:
            unique_val = np.sort(df_x[c].unique())
            self.train_unique_rankgauss[c]= [unique_val, self.rank_gauss(unique_val)]
        self.fit_done = True

    def transform(self, df_target):
        """
        df_target: transform対象のDataFrame
        """
        assert self.fit_done
        assert np.all(np.sort(np.intersect1d(df_target.columns, self.target_cols)) == np.sort(self.target_cols))
        df_converted_rank_gauss = pd.DataFrame(index=df_target.index)
        for c in self.target_cols:
            df_converted_rank_gauss[c] = np.interp(df_target[c], 
                                                   self.train_unique_rankgauss[c][0], 
                                                   self.train_unique_rankgauss[c][1]) # ,left=0, right=0)
        return df_converted_rank_gauss

In [5]:
from torchvision import transforms

def worker_init_fn(worker_id):                                                          
    np.random.seed(np.random.get_state()[1][0] + worker_id)

class plasticcDataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.from_numpy(x.astype(np.float32))
#        self.y = torch.from_numpy(y.astype(np.float32))
        self.y = torch.LongTensor(y.astype(np.float32))
        #self.y = y
#        self.transform = transforms.Compose([transforms.ToTensor()])
 
    def __len__(self):
        return self.x.shape[0]
 
    def __getitem__(self, idx):
        return self.x[idx, :], self.y[idx]

In [6]:
# objective
# y_h に nan が起こりうる -> classsize が　0 になりうるのでどうにかせねば...
# 
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
def wloss_metric(preds, train_data):
    weight_tensor = torch.tensor(np.array(list(class_weight.values())).astype(np.float), requires_grad=True).type(torch.FloatTensor)
    #weight_tensor = torch.tensor(list(class_weight.values()), requires_grad=True).type(torch.FloatTensor)
    y_t = torch.tensor(train_data, requires_grad=True).type(torch.LongTensor)
    y_h = torch.zeros(y_t.shape[0], len(classes), requires_grad=True).scatter(1, y_t.reshape(-1, 1), 1)
    print(y_t.shape)
    y_h[y_h == 0] = 1
    y_h /= y_h.sum(dim=0, keepdim=True)
    y_p = torch.tensor(preds, requires_grad=True).type(torch.FloatTensor)
    if len(y_p.shape) == 1:
        y_p = y_p.reshape(len(classes), -1).transpose(0, 1)
    ln_p = torch.log_softmax(y_p, dim=1)
    wll = torch.sum(y_h * ln_p, dim=0)
    loss = -torch.dot(weight_tensor, wll) / torch.sum(weight_tensor)
    return loss

def mywloss(y_pred_raw, y_true):  
    weight_tensor = torch.tensor(np.array(list(class_weight.values())).astype(np.float), requires_grad=True).type(torch.FloatTensor)
    y_onehot = torch.FloatTensor(y_true.shape[0], len(classes))
    y_onehot.zero_()
    y_onehot.scatter_(1, y_true.reshape(-1, 1), 1)
    
    y_pred_log=torch.log_softmax(y_pred_raw, dim=1)
    class_num = torch.sum(y_onehot, dim=0)
    class_num[class_num == 0] = 1
    loss=-(torch.mean(torch.sum(torch.sum(y_onehot*y_pred_log, dim=0)*weight_tensor/class_num) / torch.sum(weight_tensor)))
    return loss

def mywloss2(y_pred_raw, y_true):  
    weight_tensor = torch.tensor(np.array(list(class_weight.values())).astype(np.float), requires_grad=True).type(torch.FloatTensor)
    y_onehot = torch.FloatTensor(y_true.shape[0], len(classes))
    y_onehot.zero_()
    y_onehot.scatter_(1, y_true.reshape(-1, 1), 1)
    
    y_pred_log=torch.log_softmax(y_pred_raw, dim=1)
    class_num = torch.sum(y_onehot, dim=0)
    class_num[class_num == 0] = 1
######    #loss=-(torch.mean(torch.sum(torch.sum(y_onehot*y_pred_log, dim=0)*weight_tensor/class_num) / torch.sum(weight_tensor)))
    loss=-(torch.mean(torch.sum(torch.sum(y_onehot*y_pred_log, dim=0)*weight_tensor)/torch.sum(weight_tensor)))
    return loss

def get_l2_loss(model, weight=0.1):
    l2_weight = torch.tensor(weight)
    l2_reg = torch.tensor(0.)
    for param in model.parameters():
        l2_reg += torch.norm(param)
    return l2_weight * l2_reg

def calc_team_score(y_true, y_preds):
    '''
    y_true:$B#1<!85$N(Bnp.array
    y_pred:softmax$B8e$N#1(B4$B<!85$N(Bnp.array
    '''
    y_preds =  torch.softmax(y_preds, dim=1).detach().numpy()
    
    class99_prob = 1/9
    class99_weight = 2
            
    y_p = y_preds * (1-class99_prob)
    y_p = np.clip(a=y_p, a_min=1e-15, a_max=1 - 1e-15)
    y_p_log = np.log(y_p)
    
    y_true_ohe = pd.get_dummies(y_true).values
    nb_pos = y_true_ohe.sum(axis=0).astype(float)
    
    classes = [6, 15, 16, 42, 52, 53, 62, 64, 65, 67, 88, 90, 92, 95]
    class_weight = {6: 1, 15: 2, 16: 1, 42: 1, 52: 1, 53: 1, 62: 1, 64: 2, 65: 1, 67: 1, 88: 1, 90: 1, 92: 1, 95: 1}
    class_arr = np.array([class_weight[k] for k in sorted(class_weight.keys())])
    
    y_log_ones = np.sum(y_true_ohe * y_p_log, axis=0)
    y_w = y_log_ones * class_arr / nb_pos
    score = - np.sum(y_w) / (np.sum(class_arr)+class99_weight)\
        + (class99_weight/(np.sum(class_arr)+class99_weight))*(-np.log(class99_prob))

    return score

In [24]:
%%time
#imp_df = pd.read_csv('../importances/Booster_weight-multi-logloss-0.551369_2018-12-09-13-05-52_importance.csv')
#imp_df = pd.read_csv('../importances/Booster_weight-multi-logloss-0.495417_2018-12-14-18-47-29_importance.csv')
print('========== 0 =========')
#imp_df = pd.read_csv('../importances/Booster_weight-multi-logloss-0.534367_2018-12-15-18-49-06_importance.csv')
imp_df = pd.read_csv('../importances/Booster_weight-multi-logloss-0.581929_2018-12-26-13-15-24_importance.csv')
FEATURES_TO_USE = imp_df.head(120).feature.tolist()

print('========== 1 =========')
train_df = pd.read_feather('../features/train/meta_features.ftr', nthreads=NTHREAD)
test_df = pd.read_feather('../features/test/meta_features.ftr', nthreads=NTHREAD)
#train_df = pd.read_pickle('../features/onodera_feats/X_train_0_1217-1.pkl.gz', compression='gzip')
#test_df = pd.read_pickle('../features/onodera_feats/X_test_0_1217-1.pkl.gz', compression='gzip')

print('========== 2 =========')
# inf の変換
train_df = train_df.replace(np.inf, np.nan)
train_df = train_df.replace(-np.inf, np.nan)

test_df = test_df.replace(np.inf, np.nan)
test_df = test_df.replace(-np.inf, np.nan)

#train_df = train_df[FEATURES_TO_USE]
#test_df = test_df[FEATURES_TO_USE]
train_df = train_df[list(set(test_df.columns.tolist()) & set(train_df.columns.tolist()))]

print('========== 3 =========')
mkscaler = MKRankGaussScalar()

print('========== 4 =========')
mkscaler.fit(train_df)
x_train = mkscaler.transform(train_df).values
x_test = mkscaler.transform(test_df).values

print('========== 5 =========')
x_train_scaled = x_train
x_train_scaled = pd.DataFrame(x_train_scaled).fillna(0.).values
gc.collect()

# 欠損値埋め
x_test_scaled = x_test
x_test_scaled = pd.DataFrame(x_test_scaled).fillna(0.).values
gc.collect()

========== 0 =========
========== 1 =========
========== 2 =========
========== 3 =========
========== 4 =========
========== 5 =========
CPU times: user 22min 7s, sys: 14min 30s, total: 36min 37s
Wall time: 32min 15s


In [18]:
x_train.shape

(271148, 120)

In [25]:
x_train.shape

(271148, 1605)

In [26]:
label_num = 14
FOLD_NUM = 5
DO_RATE = 0.5
max_layer_size = 512
#SEEDS = [71, 1000, 0, 99, 713]
SEEDS = [71, ]


le = LabelEncoder()
#le.fit(train_df['target'].values)
#read_targets = pd.read_csv('/home/naoya.taguchi/.kaggle/competitions/PLAsTiCC-2018/training_set_metadata.csv')
read_targets = pd.read_hdf('/home/naoya.taguchi/.kaggle/competitions/PLAsTiCC-2018/kyle_final_augment.h5', 'meta').reset_index()
le.fit(read_targets['target'].values)

x_train = x_train_scaled
#y_train = le.transform(train_df.target)
y_train = le.transform(read_targets.target)


#seed = 90
for seed in SEEDS:
    torch.manual_seed(seed)

    # skf = StratifiedKFold(n_splits=FOLD_NUM, shuffle=True, random_state=71)
    # folds = skf.split(x_train, y_train)
    folds = []
    for fold in range(5):
        _trn_idx = read_targets.query(f'fold != {fold}').index.tolist()
        _val_idx = read_targets.query(f'fold == {fold}').index.tolist()
        folds.append([_trn_idx, _val_idx])

    best_models = []
    best_scores = []
    best_team_scores = []
    best_losses = []
    oof = []

    for trn_idx, val_idx in tqdm(list(folds)):
        x_trn, x_val = x_train[trn_idx], x_train[val_idx]
        y_trn, y_val = y_train[trn_idx], y_train[val_idx]
    
        #mlp_unit_nums = (max_layer_size, max_layer_size//2, max_layer_size//4,  label_num)
        mlp_unit_nums = (max_layer_size, max_layer_size//2, max_layer_size//4, max_layer_size//8, label_num)
    
        pnn = plasticcNet(input_unit_num=x_trn.shape[1], mlp_unit_nums=mlp_unit_nums, init_do_rate=DO_RATE*0.2, middle_do_rate=DO_RATE, last_do_rate=DO_RATE*0.5)
        optimizer = optim.SGD(pnn.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4, nesterov=True)
        # optimizer = optim.SGD(pnn.parameters(), lr=0.6, momentum=0.0)
        # scheduler = StepLR(optimizer, step_size=5, gamma=0.6)
            
        dataset = plasticcDataset(x_trn, y_trn)
        dataloader = DataLoader(dataset, batch_size=100, shuffle=True, num_workers=0, worker_init_fn=worker_init_fn)
    
        epochs = 1000
        losses = []
        best_val = 100000000
        best_count = 0
        best_model = None   
        
        for epoch in range(epochs):
            # scheduler.step()
            for i_batch, sample_batched in enumerate(dataloader):
                pnn = pnn.train()
                optimizer.zero_grad()
                x_trn_batch, y_trn_batch = sample_batched
                x_trn_batch, y_trn_batch = Variable(x_trn_batch), Variable(y_trn_batch)
                _res = pnn(x_trn_batch)
                loss = mywloss(_res, y_trn_batch)
                #loss = mywloss2(_res, y_trn_batch)
                #loss = cross_entropy(_res, y_trn_batch)
                loss.backward()
                optimizer.step()
                losses.append(loss.detach().numpy())
                #break
            pnn = pnn.eval()
            # oof いれる！！！！
            val_res = pnn(torch.Tensor(x_val))
            val_score = mywloss(val_res, torch.LongTensor(y_val)).detach().numpy()
            #val_score = cross_entropy(val_res, torch.LongTensor(y_val)).detach().numpy()
            val_team_score = calc_team_score(y_val, val_res)
            if epoch % 1 == 0:
                if best_val > val_score:
                    best_epoch = epoch
                    best_val = val_score
                    best_team_val = val_team_score
                    best_loss = np.mean(losses)
                    best_count = 0
                    torch.save(pnn.state_dict(), '../check_points/temp.pth')
                else:
                    best_count += 1
                print(f'epoch : {epoch}', f'lr: {optimizer.state_dict()["param_groups"][0]["lr"]:.5f}', f'train_loss : {np.mean(losses):.5f}', f'val_loss : {val_score:.5f}', f'akiyama-metric{best_team_val:.5f}')
                losses = []
                if best_count >= 30:
                    print(f'best epoch: {best_epoch}', f'best training loss {best_loss:.5f}', f'best val: {best_val:.5f}', f'best team val: {best_team_val:.5f}')
                    pnn.load_state_dict(torch.load('../check_points/temp.pth'))
                    best_models.append(pnn)
                    best_val_res = pnn(torch.Tensor(x_val))
                    oof.append([best_val_res.detach().numpy(), y_val, val_idx])
                    best_scores.append(best_val)
                    best_team_scores.append(best_team_val)
                    best_losses.append(best_loss)
                    gc.collect()
                    break
            # break
    #    break
    
    submission_file_name = '../submissions/PLASTICC_NET_weight-multi-logloss-{:.6}.csv'.format(np.mean(best_scores))
    print(f'===== {np.mean(best_scores)} =====')
    
    oof_filename = submission_file_name.split('/')[-1][:-4]
    oof_filename = '../oof/' + oof_filename + '.pkl'
    with open(oof_filename, 'wb') as fout:
        pickle.dump(oof, fout)
    print(f'saved oof to {oof_filename}')   
    
    test_reses = []
    for bpnn in tqdm(best_models):
        # torch の nn の出力を ↑　の自作 softmax でさばくと nan ができてしまう
        test_reses.append(torch.nn.functional.softmax(bpnn(torch.Tensor(x_test_scaled))).detach().numpy())
    
    res = np.clip(np.mean(test_reses, axis=0), 10**(-15), 1 - 10**(-15))
    preds_99 = np.ones((res.shape[0]))
    for i in range(res.shape[1]):
        preds_99 *= (1 - res[:, i])
    preds_99 = 0.14 * preds_99 / np.mean(preds_99)
    
    res_df = pd.DataFrame(res, columns=[
        'class_6',
        'class_15',
        'class_16',
        'class_42',
        'class_52',
        'class_53',
        'class_62',
        'class_64',
        'class_65',
        'class_67',
        'class_88',
        'class_90',
        'class_92',
        'class_95',
    ])
    res_df['class_99'] = preds_99
    
    object_ids = pd.read_csv('/home/naoya.taguchi/.kaggle/competitions/PLAsTiCC-2018/test_set_metadata.csv').object_id
    pd.concat([object_ids, res_df], axis=1).to_csv(submission_file_name, index=False)

HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

epoch : 0 lr: 0.01000 train_loss : 0.75159 val_loss : 0.60484 akiyama-metric0.88646
epoch : 1 lr: 0.01000 train_loss : 0.57259 val_loss : 0.53308 akiyama-metric0.82268
epoch : 2 lr: 0.01000 train_loss : 0.52658 val_loss : 0.51688 akiyama-metric0.80828
epoch : 3 lr: 0.01000 train_loss : 0.50022 val_loss : 0.49199 akiyama-metric0.78616
epoch : 4 lr: 0.01000 train_loss : 0.47760 val_loss : 0.49634 akiyama-metric0.78616
epoch : 5 lr: 0.01000 train_loss : 0.45862 val_loss : 0.47455 akiyama-metric0.77065
epoch : 6 lr: 0.01000 train_loss : 0.44424 val_loss : 0.48664 akiyama-metric0.77065
epoch : 7 lr: 0.01000 train_loss : 0.43716 val_loss : 0.47020 akiyama-metric0.76678
epoch : 8 lr: 0.01000 train_loss : 0.42198 val_loss : 0.48117 akiyama-metric0.76678
epoch : 9 lr: 0.01000 train_loss : 0.41542 val_loss : 0.46013 akiyama-metric0.75784
epoch : 10 lr: 0.01000 train_loss : 0.40991 val_loss : 0.46182 akiyama-metric0.75784
epoch : 11 lr: 0.01000 train_loss : 0.40265 val_loss : 0.45774 akiyama-metr

epoch : 14 lr: 0.01000 train_loss : 0.38531 val_loss : 0.46194 akiyama-metric0.75944
epoch : 15 lr: 0.01000 train_loss : 0.37750 val_loss : 0.49047 akiyama-metric0.75944
epoch : 16 lr: 0.01000 train_loss : 0.37097 val_loss : 0.46692 akiyama-metric0.75944
epoch : 17 lr: 0.01000 train_loss : 0.36586 val_loss : 0.46913 akiyama-metric0.75944
epoch : 18 lr: 0.01000 train_loss : 0.36260 val_loss : 0.46898 akiyama-metric0.75944
epoch : 19 lr: 0.01000 train_loss : 0.35878 val_loss : 0.47431 akiyama-metric0.75944
epoch : 20 lr: 0.01000 train_loss : 0.35781 val_loss : 0.49185 akiyama-metric0.75944
epoch : 21 lr: 0.01000 train_loss : 0.35175 val_loss : 0.48184 akiyama-metric0.75944
epoch : 22 lr: 0.01000 train_loss : 0.35082 val_loss : 0.46458 akiyama-metric0.75944
epoch : 23 lr: 0.01000 train_loss : 0.34841 val_loss : 0.47957 akiyama-metric0.75944
epoch : 24 lr: 0.01000 train_loss : 0.34072 val_loss : 0.47210 akiyama-metric0.75944
epoch : 25 lr: 0.01000 train_loss : 0.34428 val_loss : 0.47870 ak

epoch : 0 lr: 0.01000 train_loss : 0.74904 val_loss : 0.55357 akiyama-metric0.84090
epoch : 1 lr: 0.01000 train_loss : 0.57221 val_loss : 0.52055 akiyama-metric0.81154
epoch : 2 lr: 0.01000 train_loss : 0.52351 val_loss : 0.50430 akiyama-metric0.79710
epoch : 3 lr: 0.01000 train_loss : 0.49606 val_loss : 0.50209 akiyama-metric0.79513
epoch : 4 lr: 0.01000 train_loss : 0.47114 val_loss : 0.48630 akiyama-metric0.78110
epoch : 5 lr: 0.01000 train_loss : 0.45710 val_loss : 0.48969 akiyama-metric0.78110
epoch : 6 lr: 0.01000 train_loss : 0.44518 val_loss : 0.47290 akiyama-metric0.76919
epoch : 7 lr: 0.01000 train_loss : 0.42633 val_loss : 0.47942 akiyama-metric0.76919
epoch : 8 lr: 0.01000 train_loss : 0.42072 val_loss : 0.47676 akiyama-metric0.76919
epoch : 9 lr: 0.01000 train_loss : 0.41006 val_loss : 0.46394 akiyama-metric0.76123
epoch : 10 lr: 0.01000 train_loss : 0.40194 val_loss : 0.47051 akiyama-metric0.76123
epoch : 11 lr: 0.01000 train_loss : 0.39581 val_loss : 0.47080 akiyama-metr

epoch : 48 lr: 0.01000 train_loss : 0.30629 val_loss : 0.49951 akiyama-metric0.76808
epoch : 49 lr: 0.01000 train_loss : 0.30123 val_loss : 0.49114 akiyama-metric0.76808
epoch : 50 lr: 0.01000 train_loss : 0.29978 val_loss : 0.48816 akiyama-metric0.76808
epoch : 51 lr: 0.01000 train_loss : 0.29743 val_loss : 0.50988 akiyama-metric0.76808
epoch : 52 lr: 0.01000 train_loss : 0.29802 val_loss : 0.48995 akiyama-metric0.76808
epoch : 53 lr: 0.01000 train_loss : 0.29696 val_loss : 0.51274 akiyama-metric0.76808
epoch : 54 lr: 0.01000 train_loss : 0.29623 val_loss : 0.51161 akiyama-metric0.76808
epoch : 55 lr: 0.01000 train_loss : 0.29317 val_loss : 0.48958 akiyama-metric0.76808
epoch : 56 lr: 0.01000 train_loss : 0.29269 val_loss : 0.48386 akiyama-metric0.76808
epoch : 57 lr: 0.01000 train_loss : 0.29244 val_loss : 0.49288 akiyama-metric0.76808
epoch : 58 lr: 0.01000 train_loss : 0.29095 val_loss : 0.49665 akiyama-metric0.76808
epoch : 59 lr: 0.01000 train_loss : 0.28802 val_loss : 0.50477 ak

HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

In [ ]:
pd.read_pickle('../share/PLASTICC_NET_weight-multi-logloss-0.538699.pkl')

In [12]:
PLASTICC_NET_weight-multi-logloss-0.59564.pkl

NameError: name 'PLASTICC_NET_weight' is not defined

In [27]:
lgb_df = pd.read_csv('../submissions/Booster_weight-multi-logloss-0.581929_2018-12-26-13-15-24.csv')
nn_df = pd.read_csv('../submissions/PLASTICC_NET_weight-multi-logloss-0.59564.csv')

In [31]:
((lgb_df.set_index('object_id') + nn_df.set_index('object_id'))/2).to_csv('../submissions/nn_and_lgb_kyle.csv.gz', index=True, compression='gzip')